### Importing libraries 

In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report


### Define the path to the dataset

In [24]:
data_dir = 'cell_images'

### Retrieve the class names from the subdirectories in the dataset directory.

In [25]:
class_names = sorted(os.listdir(data_dir))

 ## split the dataset into training and validation and load 

In [26]:
batch_size = 32
img_height = 100
img_width = 100

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.



### Preprocess the data

In [27]:
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))


### Configuring the datasets for performance by caching and prefetching.



In [28]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Building the model

In [29]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names))
])

### Compiling the model

In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


### Training the mode

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 628s 904ms/step - accuracy: 0.7891 - loss: 0.4198 - val_accuracy: 0.9423 - val_loss: 0.1796
Epoch 2/10
196/689 ━━━━━━━━━━━━━━━━━━━━ 4:46 582ms/step - accuracy: 0.9518 - loss: 0.1602

## Plot training history
 1. training & validation accuracy values
 2. training & validation loss values


In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()



### Evaluating the trained model on the validation dataset

In [ ]:
val_loss, val_accuracy = model.evaluate(val_ds)
print("Validation accuracy:", val_accuracy)



### Generate predictions

In [ ]:
y_pred = np.argmax(model.predict(val_ds), axis=-1)
y_true = np.concatenate([y for x, y in val_ds], axis=0)

### Printing the classification report

In [ ]:
print(classification_report(y_true, y_pred, target_names=class_names))